# Residential Floor Heights

Data ingestion example.


## Database setup

- Deploy the docker services and migrate postgres to the latest revision of the data model.

In [ ]:
# Deploy the docker services
!docker compose up -d

# Migrate to latest revision of data model
!docker compose run --rm app alembic upgrade head

## Database ingestions

- Run scripts to ingest local data, this will take a couple of minutes.

In [64]:
# Wagga Wagga data ingestion

# File path to sharepoint project directory
SHAREPOINT="/home/lng/sharepoint/Projects - Documents/Projects - Data Analytics/127 Residential Dwelling Floor Height/4 Executing"

# Set up file paths
ADDRESS=SHAREPOINT+"/Data Exploration/national_address_points/wagga_address_points.gdb/"
BUILDINGS=SHAREPOINT+"/Data Exploration/overture/overture_buildings_australia.parquet"
DEM=SHAREPOINT+"/Data Exploration/Wagga DEM/NSW_2020_DEM_1m.vrt"
CADASTRE=SHAREPOINT+"/Data Exploration/nsw_cadastre/Wagga_Lot_EPSG7844.gdb"
NEXIS=SHAREPOINT+"/GA_data_documentation/nexis_building_data/NSW_building.csv"
VALIDATION=SHAREPOINT+"/GA_data_documentation/Wagga Exposure Data/Final_Wagga.UFI/Final_Wagga_floor_heights.gpkg"
OUTPUT="wagga_wagga.gpkg"

# Ingest the address points
!python ../floorheights/datamodel/cli.py ingest-address-points -i "{ADDRESS}"

# Ingest the building footprints
!python ../floorheights/datamodel/cli.py ingest-buildings -i "{BUILDINGS}" -d "{DEM}" --split-by-cadastre "{CADASTRE}" --remove-overlapping --remove-small

# Join address points to building footprints
!python ../floorheights/datamodel/cli.py join-address-buildings -c "{CADASTRE}" --join-largest-building

# Ingest NEXIS method floor measures
!python ../floorheights/datamodel/cli.py ingest-nexis-method -i "{NEXIS}" -c "{CADASTRE}" --join-largest-building --clip-to-cadastre

# Ingest Validation method floor measures
!python ../floorheights/datamodel/cli.py ingest-validation-method -i "{VALIDATION}" --ffh-field "floor_height_m" -c "{CADASTRE}" --join-largest-building

# Export a geopackage of the data model
!python ../floorheights/datamodel/cli.py export-ogr-file -o "{OUTPUT}"


Loading Geodatabase...
Copying to PostgreSQL...
Address ingestion complete
Loading DEM...
Creating mask...
Loading building GeoParquet...
Splitting buildings by cadastre...
Removing buildings < 30.0 m^2...
Removed 11807 buildings...
Sampling DEM with buildings...
Copying to PostgreSQL...
Removing overlapping buildings...
Removed 54 overlapping buildings...
Building ingestion complete
Performing join by knn...
Loading cadastre...
Copying cadastre to PostgreSQL...
Performing join with cadastre...
Joining with largest building on lot...
Joining complete
Loading NEXIS points...
Clipping NEXIS points to cadastre...
Copying NEXIS points to PostgreSQL...
Inserting GNAF records into floor_measure table...
Inserting non-GNAF records into floor_measure table...
Copying cadastre to PostgreSQL...
Joining with largest building on lot...
NEXIS ingestion complete
Loading validation points...
Copying validation points to PostgreSQL...
Copying cadastre to PostgreSQL...
Inserting records into floor_meas

## Stop and clear database

- Stop the docker containers and remove the docker volume to clear the data.
- Comment the top line of cell below to run.

In [ ]:
%%script echo skipping cell  # Comment this line to run this cell

# Stop all containers, removing the postgres_data volume
!docker compose down
!docker volume rm ga-floor-height-data-model_postgres_data